# HSE 2022: Mathematical Methods for Data Analysis

## Homework 2

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import datasets
from sklearn.datasets import load_boston
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
from math import sqrt
import random
import sys

import warnings

warnings.filterwarnings("ignore")

%matplotlib inline

sns.set(style="darkgrid")

### Data

For this homework we use Dataset from seaborn on diamonds prices.

In [ ]:
data = sns.load_dataset('diamonds')
y = data.price
X = data.drop(['price'], axis=1)
columns = data.drop(['price'], axis=1).columns

## Linear regression

#### 0. [0.25 points] Encode categorical variables.

In [ ]:
categories = (data.dtypes == "category")
object_cols = list(categories[categories].index)

encoded_data = X.copy()
label_encoder = LabelEncoder()
for col in object_cols:
    encoded_data[col] = label_encoder.fit_transform(encoded_data[col])

#### 1. [0.25 points] Split the data into train and test sets with ratio 80:20 with random_state=17.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded_data, y, test_size=0.2, random_state=17)

#### 2. [1 point] Train models on train data using StatsModels library and apply it to the test set; use $RMSE$ and $R^2$ as the quality measure.

* [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html);
* [`Ridge`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) with $\alpha = 0.01$;
* [`Lasso`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) with $\alpha = 0.01$
* [`ElasticNet`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html) with $\alpha = 0.01$, $l_{1}$_$ratio = 0.6$

Don't forget to scale the data before training the models with StandardScaler!

In [ ]:
# scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
# for statsmodels
X_train = sm.add_constant(X_train)
X_test = sm.add_constant(X_test)
model = sm.OLS(y_train, X_train)

# Linear Regression from statsmodels
results_lr = model.fit()
y_test_predicted = results_lr.predict(X_test)
y_train_predicted = results_lr.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Ridge from statsmodels
results_ridge = model.fit_regularized(L1_wt=0, alpha=0.01)
y_test_predicted = results_ridge.predict(X_test)
y_train_predicted = results_ridge.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Lasso from statsmodels
results_lasso = model.fit_regularized(L1_wt=1, alpha=0.01)
y_test_predicted = results_lasso.predict(X_test)
y_train_predicted = results_lasso.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Elastic net from statsmodels
results_elastic = model.fit_regularized(L1_wt=0.6, alpha=0.01)
y_test_predicted = results_elastic.predict(X_test)
y_train_predicted = results_elastic.predict(X_train)

print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

The best quality has Linear Regression model because it has the smallest RMSE and the highest R2 score

#### 3. [1 point] Explore the values of the parameters of the resulting models and compare the number of zero weights in them. Comment on the significance of the coefficients, overal model significance and other related factors from the results table

In [ ]:
features = list(X.iloc[:, :10].columns)
features.insert(0, "const")

# linear regression
results_lr.summary2(xname=features)

The values of the features are in "coef" column. There is no zero weights.
Linear Regression summary:
    We consider significance level = 0.05.
   1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
   2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
   3. R2_adj show that ~88.5% of the variance is explained, so the model is valuable
   4. As F-statistics ia a pretty large number, we can say that the model is significant
   5. Durbin-Watson value is nearly 2, so there is no autocorrelation
   6. Large Jarque-Bera value indicates that errors are not normally distributed

In [ ]:
# ridge
OLSResults(model, results_ridge.params, model.normalized_cov_params).summary2(xname=features)

The values of the features are in "coef" column. There is no zero weights.
Ridge Regression summary:
    We consider significance level = 0.05.
   1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
   2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
   3. R2_adj show that ~88.2% of the variance is explained, so the model is valuable
   4. As F-statistics ia a pretty large number, we can say that the model is significant
   5. Durbin-Watson value is nearly 2, so there is no autocorrelation
   6. Large Jarque-Bera value indicates that errors are not normally distributed

In [ ]:
# lasso
OLSResults(model, results_lasso.params, model.normalized_cov_params).summary2(xname=features)

The values of the features are in "coef" column. There is no zero weights.
Lasso Regression summary:
    We consider significance level = 0.05.
   1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
   2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
   3. R2_adj show that ~88.5% of the variance is explained, so the model is valuable
   4. As F-statistics ia a pretty large number, we can say that the model is significant
   5. Durbin-Watson value is nearly 2, so there is no autocorrelation
   6. Large Jarque-Bera value indicates that errors are not normally distributed

In [ ]:
# elastic
OLSResults(model, results_elastic.params, model.normalized_cov_params).summary2(xname=features)

The values of the features are in "coef" column. There is no zero weights.
Elastic Net Regression summary:
    We consider significance level = 0.05.
   1. P-values of "y" is higher than 0.05, so we can call "y" coefficient insignificant
   2. P-values of "z" is higher than 0.05, so we can call "z" coefficient insignificant
   3. R2_adj show that ~88.4% of the variance is explained, so the model is valuable
   4. As F-statistics ia a pretty large number, we can say that the model is significant
   5. Durbin-Watson value is nearly 2, so there is no autocorrelation
   6. Large Jarque-Bera value indicates that errors are not normally distributed

<bold> Summary: <bold>
1. As Linear Regression model has the smallest AIC value, it should be selected.
2. Linear Regression model also has the smallest BIC value
3. In all models "y" and "z" are insignificant
4. The Ridge model is the most skewed

#### 4. [1 point] Implement one of the elimination algorithms that were described in the Seminar_4 (Elimination by P-value, Forward elimination, Backward elimination), make conclusions.

<bold>We consider significance level = 0.05.<bold>

In [ ]:
# p-value elimination for linear regression model
model = sm.OLS(y_train, X_train)
results = model.fit()
sign_level = 0.05

features = list(X.iloc[:, :10].columns)
features.insert(0, "const")
features = np.array(features)

print("Start from model with all features:")
print("R2_adj = %.10f" % results.rsquared_adj)
print("RMSE = %.10f" % mean_squared_error(y_train, results.predict(X_train), squared=False))
print("AIC = %.10f\n" % results.aic)
p_values = np.array(results.pvalues)
X_eliminated = X_train

while np.max(p_values) > sign_level:
    insign_feature_index = np.argmax(p_values)
    X_eliminated = np.delete(X_eliminated, insign_feature_index, axis=1)
    print(f'Feature \"{features[insign_feature_index]}\" was eliminated.')
    features = np.delete(features, insign_feature_index)
    model = sm.OLS(y_train, X_eliminated)
    results = model.fit()
    p_values = np.array(results.pvalues)
    print("Current:\nR2_adj = %.10f" % results.rsquared_adj)
    print("RMSE = %.10f" % mean_squared_error(y_train, results.predict(X_eliminated), squared=False))
    print("AIC = %.10f\n" % results.aic)

print("Features after p-value elimination algorithm:")
print(features)

Summary: the elimination of the "z" increased model's quality, so we can remove it.

#### 5. [1 point] Find the best (in terms of RMSE) $\alpha$ for Lasso regression using cross-validation with 4 folds. You must select values from range $[10^{-4}, 10^{3}]$.

In [ ]:
parameters = {'alpha': np.logspace(-4, 3)}
search = GridSearchCV(Lasso(), parameters, cv=4, scoring='neg_root_mean_squared_error')
search.fit(X_train, y_train)
print(search.best_params_)

## Gradient descent

#### 6. [3.5 points] Implement a Ridge regression model for the MSE loss function, trained by gradient descent.

All calculations must be vectorized, and python loops can only be used for gradient descent iterations. As a stop criterion, you must use (simultaneously):

* checking for the Absolute-value norm of the weight difference on two adjacent iterations (for example, less than some small number of the order of $10^{-6}$, set by the `tolerance` parameter);
* reaching the maximum number of iterations (for example, 10000, set by the `max_iter` parameter).

You need to implement:

* Full gradient descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} Q(w_{k}).
$$

* Stochastic Gradient Descent:

$$
w_{k + 1} = w_{k} - \eta_{k} \nabla_{w} q_{i_{k}}(w_{k}).
$$

$\nabla_{w} q_{i_{k}}(w_{k}) \, $ is the estimate of the gradient over the batch of objects selected randomly.

* Momentum method:

$$
h_0 = 0, \\
h_{k + 1} = \alpha h_{k} + \eta_k \nabla_{w} Q(w_{k}), \\
w_{k + 1} = w_{k} - h_{k + 1}.
$$

* Adagrad method:

$$
G_0 = 0, \\
G_{k + 1} = G_{k} + (\nabla_{w} Q(w_{k+1}))^2, \\
w_{k + 1} = w_{k} - \eta * \frac{\nabla_{w} Q(w_{k+1})}{\sqrt{G_{k+1} + \epsilon}}.
$$



To make sure that the optimization process really converges, we will use the `loss_history` class attribute. After calling the `fit` method, it should contain the values of the loss function for all iterations, starting from the first one (before the first step on the anti-gradient).

You need to initialize the weights with a random vector from normal distribution. The following is a template class that needs to contain the code implementing all variations of the models.

In [ ]:
from sklearn.base import BaseEstimator
from numpy import linalg as la


class LinReg(BaseEstimator):
    def __init__(self, delta=1.0, gd_type='Momentum',
                 tolerance=1e-4, max_iter=1000, w0=None, eta=1e-2, alpha=1e-3, reg_cf=1.0, epsilon=10e-8):
        """
        gd_type: str
            'GradientDescent', 'StochasticDescent', 'Momentum', 'Adagrad'
        delta: float
            proportion of object in a batch (for stochastic GD)
        tolerance: float
            for stopping gradient descent
        max_iter: int
            maximum number of steps in gradient descent
        w0: np.array of shape (d)
            init weights
        eta: float
            learning rate
        alpha: float
            momentum coefficient
        reg_cf: float
            regularization coefficient
        epsilon: float
            numerical stability
        """

        self.delta = delta
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.loss_history = None  # list of loss function values at each training iteration
        self.epsilon = epsilon
        self.reg_cf = reg_cf

    def fit(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: self
        """
        self.loss_history = []
        X = np.insert(X, 0, 1, axis=1)
        if self.gd_type == 'GradientDescent':
            self.w = self.full_grad_descent_calc(X, y)
        if self.gd_type == 'Momentum':
            self.w = self.momentum_descent_calc(X, y)
        if self.gd_type == 'Adagrad':
            self.w = self.adagrad_descent_calc(X, y)
        if self.gd_type == 'StochasticDescent':
            self.w = self.stochastic_grad_descent_calc(X, y)
        return self

    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return np.dot(np.insert(X, 0, 1, axis=1), self.w)

    def calc_gradient(self, X, y, w):
        """
        X: np.array of shape (l, d) (l can be equal to 1 if stochastic)
        y: np.array of shape (l)
        ---
        output: np.array of shape (d)
        """
        return 2 * np.dot(X.T, np.dot(X, w) - y) / np.shape(X)[0] + 2 * self.reg_cf * w / np.shape(X)[0]

    def full_grad_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 0:
            gradient = self.calc_gradient(X, y, w)
            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            self.w = w
            w -= self.eta * gradient
            iter_num += 1
        return w

    def momentum_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        h = np.zeros(np.shape(X)[1])
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 0:
            gradient = self.calc_gradient(X, y, w)
            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            h = self.alpha * h + self.eta * gradient
            self.w = w
            w -= h
            iter_num += 1
        return w

    def adagrad_descent_calc(self, X, y):
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        g = np.zeros(np.shape(X)[1])
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 0:
            gradient = self.calc_gradient(X, y, w)
            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            g += np.square(gradient)
            self.w = w
            w -= self.eta * np.divide(gradient, (np.sqrt(g + self.epsilon)))
            iter_num += 1
        return w

    def stochastic_grad_descent_calc(self, X, y):
        np.random.seed(42)
        w = np.random.normal(0, 1, np.shape(X)[1])
        self.w = w.copy()
        iter_num = 0
        batch_size = int(np.round(np.shape(X)[0] * self.delta))
        while iter_num < self.max_iter and abs(la.norm(w - self.w) - self.tolerance) > 0:
            # get samples from data randomly
            batch_indices = np.random.randint(X.shape[0], size=batch_size)
            X_batch = X[batch_indices, :]
            y_batch = y.iloc[batch_indices]
            gradient = self.calc_gradient(X_batch, y_batch, w)
            self.w = w
            w -= self.eta * gradient

            # if we want to run code with epochs (than max_iter = number of epochs)
            # for start in range(0, np.shape(X)[0], batch_size):
            #     stop = start + batch_size
            #     X_batch = X[start:stop]
            #     y_batch = y[start:stop]
            #     gradient = self.calc_gradient(X_batch, y_batch, w)
            #     self.w = w
            #     w -= self.eta * gradient

            self.loss_history = np.append(self.loss_history, self.calc_loss(X, y))
            iter_num += 1
        return w

    def calc_loss(self, X, y):
        """
        X: np.array of shape (l, d)
        y: np.array of shape (l)
        ---
        output: float 
        """
        return np.sum(np.square(np.dot(X, self.w) - y)) / np.shape(X)[0] + self.reg_cf * np.sum(np.square(self.w)) / np.shape(X)[0]

#### 7. [1 points] Train and validate "hand-written" models on the same data, and compare the quality with the Sklearn or StatsModels methods. Investigate the effect of the `max_iter` and `alpha` parameters on the optimization process. Is it consistent with your expectations?

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded_data, y, test_size=0.2, random_state=17)
# scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Full gradient descent
model_full_gr = LinReg(gd_type='GradientDescent')
model_full_gr.fit(X_train, y_train)
y_train_predicted = model_full_gr.predict(X_train)
y_test_predicted = model_full_gr.predict(X_test)
print("Full gradient descent (max 1000 iterations, learning rate = 0.01): ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
model_full_gr_iter_test = LinReg(gd_type='GradientDescent', max_iter=500)
model_full_gr_iter_test.fit(X_train, y_train)
y_train_predicted = model_full_gr_iter_test.predict(X_train)
y_test_predicted = model_full_gr_iter_test.predict(X_test)
print("Full gradient descent (max 500 iterations): ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
model_full_gr_iter_test = LinReg(gd_type='GradientDescent', eta=0.001)
model_full_gr_iter_test.fit(X_train, y_train)
y_train_predicted = model_full_gr_iter_test.predict(X_train)
y_test_predicted = model_full_gr_iter_test.predict(X_test)
print("Full gradient descent (max 1000 iterations, learning rate = 0.001): ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

Summary:
1. При увеличении max_iter качество модели улучшается, при уменьшении max_iter качество модели ухудшается.

In [ ]:
# Momentum
model_momentum = LinReg(gd_type='Momentum')
model_momentum.fit(X_train, y_train)
y_train_predicted = model_momentum.predict(X_train)
y_test_predicted = model_momentum.predict(X_test)
print("Momentum: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Stochastic
model_stochastic = LinReg(gd_type='StochasticDescent', delta=0.5)
model_stochastic.fit(X_train, y_train)
y_train_predicted = model_stochastic.predict(X_train)
y_test_predicted = model_stochastic.predict(X_test)
print("Stochastic: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Adagrad
model_adagrad = LinReg(gd_type='Adagrad')
model_adagrad.fit(X_train, y_train)
y_train_predicted = model_adagrad.predict(X_train)
y_test_predicted = model_adagrad.predict(X_test)
print("Adagrad: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

In [ ]:
# Sklearn model
model = Ridge()
model.fit(X_train, y_train)
y_train_predicted = model.predict(X_train)
y_test_predicted = model.predict(X_test)
print("Ridge from Sklearn: ")
print("Train RMSE = %.4f" % mean_squared_error(y_train, y_train_predicted, squared=False))
print("Train R2 = %.4f" % r2_score(y_train, y_train_predicted))
print("Test RMSE = %.4f" % mean_squared_error(y_test, y_test_predicted, squared=False))
print("Test R2 = %.4f" % r2_score(y_test, y_test_predicted))

<bold>Comparing hand-written models with model from Sklearn:<bold>
Quality of Sklearn model is a little bit better than the hand-written one, because:
   1. Sklearn model's RMSE is less than others.
   2. R2 is higher than R2 of other models.

But hand-written models still have good quality. Sklearn model explained ~89% of the data (R2). In the same time hand-written models explained ~87% of the data, which is very close to library model. Same about RMSE values.


#### 8. [1 points] Plot graphs (on the same picture) of the dependence of the loss function value on the iteration number for Full GD, SGD, Momentum and Adagrad. Draw conclusions about the rate of convergence of various modifications of gradient descent.

Don't forget about what *beautiful* graphics should look like!

In [ ]:
plt.figure(1, figsize=(10,10))
plt.plot(model_full_gr.loss_history)
plt.plot(model_momentum.loss_history)
plt.plot(model_adagrad.loss_history)
plt.plot(model_stochastic.loss_history)
plt.legend(["Full Gradient Descent", "Momentum", "Adagrad", "Stochastic GD"])
plt.xlabel("iteration number")
plt.ylabel("loss function value")
plt.title("Loss function value and iteration number dependence")